In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import ast
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, hstack

project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA

In [ ]:
user_transactions = pd.read_excel(Path(SOURCEDATA / "Transaction_User.xlsx"))
new_user_transaction = user_transactions.drop(columns=['TrxId'])

deals_data = pd.read_excel(Path(SOURCEDATA / "Cleaned_Deals.xlsx"))
deals_data = deals_data.drop(columns=['Unnamed: 0'])

deals_embeddings = pd.read_csv(Path(SOURCEDATA / "Deals_Embeddings.csv"))
deals_embeddings['ada_embedding'] = deals_embeddings['ada_embedding'].apply(ast.literal_eval)

new_user_transaction = new_user_transaction.merge(deals_data[['ContentId', 'Categories','Deal Type']], left_on='FK_ContentId', right_on='ContentId', how='left')
new_user_transaction = new_user_transaction.drop(columns=['ContentId'])
new_user_transaction

In [ ]:
new_user_transaction['LogPointsRedeemed'] = np.log1p(new_user_transaction['PointsRedeemed'])


In [ ]:
user_item_matrix = new_user_transaction.pivot_table(index='FK_BusinessUserId', columns='FK_ContentId', values='LogPointsRedeemed',fill_value=0)


In [ ]:
frequency_user_item_matrix = new_user_transaction.pivot_table(
    index='FK_BusinessUserId', 
    columns='FK_ContentId', 
    values='PointsRedeemed', 
    aggfunc='count', 
    fill_value=0
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_points_matrix = scaler.fit_transform(user_item_matrix)
normalized_frequency_matrix = scaler.fit_transform(frequency_user_item_matrix)


In [ ]:
user_item_matrix

In [ ]:
combined_matrix = 0.5 * normalized_points_matrix + 0.5 * normalized_frequency_matrix


In [ ]:
# scaler = StandardScaler()
# new_user_transaction['PointsRedeemed'] = scaler.fit_transform(new_user_transaction[['PointsRedeemed']])

In [ ]:
sparse_user_item = csr_matrix(combined_matrix)


In [ ]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=14, regularization=0.2, iterations=20)

In [ ]:
alpha_val = 40
data_conf = (sparse_user_item * alpha_val).astype('double')
model.fit(data_conf)

In [ ]:
def recommend_items(user_id, user_item_matrix, model, n_recommendations=20):
    user_index = list(user_item_matrix.index).index(user_id)
    user_interactions = sparse_user_item[user_index]
    print(f"User Interactions for user {user_id}: {user_interactions}")
    recommendations = model.recommend(user_index, user_interactions, N=n_recommendations)
    recommended_item_indices, recommended_scores = recommendations
    recommended_item_ids = [user_item_matrix.columns[item_id] for item_id in recommended_item_indices]
    
    return recommended_item_ids, recommended_scores

In [ ]:
user_id = 976485
recommended_items = recommend_items(user_id, user_item_matrix, model)
print(f"Recommended items for user {user_id}: {recommended_items[0]}")
print(f"Scores items for user {user_id}: {recommended_items[1]}")

In [ ]:
def calculate_cosine_similarity(embedding, embeddings):
    similarities = cosine_similarity([embedding], embeddings)
    return similarities[0]

In [ ]:
def rerank_recommendations(user_id, recommended_item_ids, recommended_scores, deals_embeddings, user_item_matrix):
    
    recommended_embeddings = np.array([deals_embeddings[deals_embeddings['ContentId'] == item]['ada_embedding'].values[0] for item in recommended_item_ids])
    
    
    user_interacted_items = user_item_matrix.loc[user_id]
    user_interacted_items = user_interacted_items[user_interacted_items > 0].index.tolist()
    user_interacted_embeddings = np.array([deals_embeddings[deals_embeddings['ContentId'] == item]['ada_embedding'].values[0] for item in user_interacted_items])
    
    
    user_profile_embedding = np.mean(user_interacted_embeddings, axis=0)
    print(f"user_profile_embedding for user {user_id}: {user_profile_embedding}")
    
    cosine_similarities = calculate_cosine_similarity(user_profile_embedding, recommended_embeddings)
    
    
    combined_scores = recommended_scores + cosine_similarities
    

    reranked_indices = np.argsort(combined_scores)[::-1]
    reranked_recommendations = [recommended_item_ids[i] for i in reranked_indices]
    reranked_scores = [combined_scores[i] for i in reranked_indices]
    
    return reranked_recommendations, reranked_scores

In [ ]:
reranked_items, reranked_scores = rerank_recommendations(user_id, recommended_items[0], recommended_items[1], deals_embeddings, user_item_matrix)
print(f"Recommended items for user {user_id}: {reranked_items}")
print(f"Recommendation scores: {reranked_scores}")

In [ ]:
specific_content_id = 113834
result = deals_data[deals_data['ContentId'] == specific_content_id]

result

In [ ]:
user_last_interactions = user_item_matrix.loc[user_id]
user_interacted_items = user_last_interactions[user_last_interactions > 0].index
user_deals = deals_data[deals_data['ContentId'].isin(user_interacted_items)]
user_deals

In [ ]:
user_trs = new_user_transaction[new_user_transaction["FK_BusinessUserId"] == user_id]
user_trs

In [ ]:
115094